<a href="https://colab.research.google.com/github/pym96/Code-challenge/blob/main/competition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm.notebook import tqdm
import torch.optim as optim
import torch.nn as nn
from PIL import Image
import torchvision
import shutil
import torch
import os

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!ls

drive  sample_data


In [4]:
class MonetDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

class PhotoDataset(Dataset):
    def __init__(self, folder_path, transform=None):
        self.folder_path = folder_path
        self.transform = transform
        self.images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.jpg', '.jpeg', '.png'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image


In [5]:
# 模型权重保存路径
MODEL_SAVE_PATH = '/content/drive/MyDrive/model_weights'
if not os.path.exists(MODEL_SAVE_PATH):
    os.makedirs(MODEL_SAVE_PATH)

# 加载模型权重的函数
def load_model_weights(model, file_path):
    if os.path.isfile(file_path):
        model.load_state_dict(torch.load(file_path))
        print(f"Loaded weights from {file_path}")
    else:
        print(f"No weights found at {file_path}, starting training from scratch.")

# 保存模型权重的函数
def save_model_weights(model, file_name):
    file_path = os.path.join(MODEL_SAVE_PATH, file_name)
    torch.save(model.state_dict(), file_path)
    print(f"Saved weights to {file_path}")

In [6]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyperparameters
IMAGE_DIM = 128
num_epochs = 512
batch_size = 16
learning_rate = 1e-4
weigth_decay = 1e-3

# Define transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),

    # Data augmentation
    transforms.RandomHorizontalFlip(p=0.5),  # Randomly flip the image horizontally
    transforms.RandomRotation(degrees=15),   # Random rotation of the image
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Randomly change brightness, contrast, and saturation

    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Initialize datasets
monet_dataset = MonetDataset('/content/drive/MyDrive/monet_jpg', transform)
photo_dataset = PhotoDataset('/content/drive/MyDrive/photo_jpg', transform)

# Initialize data loaders
monet_loader = DataLoader(monet_dataset, batch_size=batch_size, shuffle=True)
photo_loader = DataLoader(photo_dataset, batch_size=batch_size, shuffle=True)

In [7]:
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True)
        )

        # Correct the input size of the linear layer to match the flattened output size
        self.linear_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(262144, 1024),  # Updated to match the output size of conv_layers
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, img):
        conv_out = self.conv_layers(img)
        validity = self.linear_layers(conv_out)
        return validity

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        # Downsample
        self.down = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True)
        )

        # Residual blocks
        self.res = nn.Sequential(
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256),
            ResidualBlock(256)
        )

        # Upsample
        self.up = nn.Sequential(
            nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.down(x)
        x = self.res(x)
        x = self.up(x)
        return x

In [8]:
# Instantiate models
D = Discriminator().to(device)
G = Generator().to(device)

# 加载预训练的权重（如果存在）
# load_model_weights(D, os.path.join(MODEL_SAVE_PATH, 'discriminator.pth'))
# load_model_weights(G, os.path.join(MODEL_SAVE_PATH, 'generator.pth'))

# Optimizers (No need for BCELoss)
d_optimizer = optim.AdamW(D.parameters(), lr=learning_rate, weight_decay=weigth_decay)
g_optimizer = optim.AdamW(G.parameters(), lr=learning_rate, weight_decay=weigth_decay)

criterion_GAN = nn.MSELoss()
criterion_cycle = nn.L1Loss()
criterion_identity = nn.L1Loss()
criterion_discriminator = nn.BCELoss()

In [9]:
# Lambda factors for different components of the loss
lambda_cycle = 10.0
lambda_identity = 0.5 * lambda_cycle

def g_loss_function(real_monet, generated_photo, reconstructed_monet, real_photo, D):
    valid = torch.ones(real_monet.size(0), 1, device=device)
    g_loss_GAN = criterion_GAN(D(generated_photo), valid)
    g_loss_cycle = criterion_cycle(reconstructed_monet, real_monet)
    identity_photo = G(real_photo)
    g_loss_identity = criterion_identity(identity_photo, real_photo)
    g_loss = g_loss_GAN + lambda_cycle * g_loss_cycle + lambda_identity * g_loss_identity
    return g_loss

def d_loss_function(real_outputs, fake_outputs):
    real_labels = torch.ones(real_outputs.size(0), 1, device=device)
    real_loss = criterion_discriminator(real_outputs, real_labels)
    fake_labels = torch.zeros(fake_outputs.size(0), 1, device=device)
    fake_loss = criterion_discriminator(fake_outputs, fake_labels)
    d_loss = real_loss + fake_loss
    return d_loss


In [10]:
for epoch in range(num_epochs):
    loader = zip(photo_loader, monet_loader)
    epoch_g_loss = 0.0
    epoch_d_loss = 0.0
    for real_photos, real_monets in loader:
        real_photos = real_photos.to(device)
        real_monets = real_monets.to(device)

        # Generator forward pass
        G.zero_grad()
        monet_style_imgs = G(real_photos)
        reconstructed_photos = G(monet_style_imgs)

        # For identity loss (optional)
        real_monets = next(iter(monet_loader)).to(device)

        # Calculate generator loss
        g_loss = g_loss_function(real_photos, monet_style_imgs, reconstructed_photos, real_monets, D)
        g_loss.backward()
        g_optimizer.step()

        # Discriminator Training
        # Randomly sample real Monet images from monet_loader for discriminator training
        D.zero_grad()

        # Discriminator outputs for real and fake images
        real_outputs = D(real_monets)
        fake_outputs = D(monet_style_imgs.detach())

        # Compute Discriminator loss
        d_loss = d_loss_function(real_outputs, fake_outputs)
        d_loss.backward()
        d_optimizer.step()

        # Accumulate losses for epoch-level logging
        epoch_g_loss += g_loss.item()
        epoch_d_loss += d_loss.item()

        # 保存模型权重
    save_model_weights(D, 'discriminator.pth')
    save_model_weights(G, 'generator.pth')

    # Print epoch-level summaries
    print(f"Epoch {epoch+1}/{num_epochs} - Generator Loss: {epoch_g_loss / len(photo_loader):.4f}, Discriminator Loss: {epoch_d_loss / len(photo_loader):.4f}")

Saved weights to /content/drive/MyDrive/model_weights/discriminator.pth
Saved weights to /content/drive/MyDrive/model_weights/generator.pth
Epoch 1/512 - Generator Loss: 0.2304, Discriminator Loss: 0.0529
Saved weights to /content/drive/MyDrive/model_weights/discriminator.pth
Saved weights to /content/drive/MyDrive/model_weights/generator.pth
Epoch 2/512 - Generator Loss: 0.1682, Discriminator Loss: 0.0631
Saved weights to /content/drive/MyDrive/model_weights/discriminator.pth
Saved weights to /content/drive/MyDrive/model_weights/generator.pth
Epoch 3/512 - Generator Loss: 0.1601, Discriminator Loss: 0.0602
Saved weights to /content/drive/MyDrive/model_weights/discriminator.pth
Saved weights to /content/drive/MyDrive/model_weights/generator.pth
Epoch 4/512 - Generator Loss: 0.1410, Discriminator Loss: 0.0820
Saved weights to /content/drive/MyDrive/model_weights/discriminator.pth
Saved weights to /content/drive/MyDrive/model_weights/generator.pth
Epoch 5/512 - Generator Loss: 0.1303, Di

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/photo_jpg/2236c207aa.jpg'

In [11]:
# Configuration for style transfer
transformed_save_dir = '/content/drive/MyDrive/images'

# Ensure the save directory exists
if not os.path.exists(transformed_save_dir):
    os.makedirs(transformed_save_dir)

G.eval()

# Process Monet images in batches and save transformed images
for i, real_photos in enumerate(photo_loader):
    real_photos = real_photos.to(device)
    with torch.no_grad():
        monet_style_imgs = G(real_photos)

    # Save each transformed image
    for j, img in enumerate(monet_style_imgs):
        save_path = os.path.join(transformed_save_dir, f'monet_style_image_{i * batch_size + j}.png')
        torchvision.utils.save_image(img, save_path)

print(f"Transformed images are saved in {transformed_save_dir}")

shutil.make_archive("/kaggle/working/images", 'zip', "/kaggle/images")

Transformed images are saved in /content/drive/MyDrive/images


'/kaggle/working/images.zip'